In [45]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from IPython.display import Image

OzESI 

FA Neutral Loss m/z

Search all TAG 37:1 - TAG 56:1 for FA 18:1 - 18:10

Search TAG40:0 - 40:n for FA 18:1 - 18:n

In [46]:
mrm_list_new = pd.read_excel('SUPPLE_2.XLS')


#loop through all sheets in SUPPLE_2.XLS and make a df of Compound Name, Parent Ion, and Product Ion
mrm_list_new = pd.read_excel('SUPPLE_2.XLS', sheet_name = None)
mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion']]
#Add underscore to middle of columns names
mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
#round Parent Ion and Product Ion to 1 decimal place
mrm_list_offical['Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(1)
mrm_list_offical['Product_Ion'] = mrm_list_offical['Product_Ion'].round(1)
#create transition column by combining Parent Ion and Product Ion with arrow between numbers
mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
#change column compound name to lipid
mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
print(mrm_list_offical.head(25))


#list of MRM to pandas dataframe

# Open Lipid File
lipid_file_mrm = pd.read_csv('List_MRMs.csv', on_bad_lines='skip', sep='\t', header=None)
#Create Headers for the dataframe
headers = ['Lipid','Transition']
lipid_file_mrm.columns = headers #Assign headers to the dataframe

#split the transition column into Q1, Arrow, Q3
start = lipid_file_mrm.Transition.str.split(expand=True) #Split the transition column into Q1, Arrow, Q3
start.columns = ['Q1','Arrow','Q3'] #Assign the column names to the new dataframe
transitions_headers = ['Q1','Arrow','Q3'] #Create a list of the column names

#Create a new dataframe of mrm transitions with columns = Lipids and Q1, Q3
df_mrm= pd.DataFrame(columns=['Lipid','Q1','Q3','Transition']) #Create a new dataframe of mrm transitions with columns = Lipids and Q1, Q3
df_mrm['Lipid'] = lipid_file_mrm['Lipid'] #Assign the Lipid column from the lipid_file_mrm to the new dataframe
df_mrm['Q1'] = start['Q1'] #    Assign the Q1 column from the start dataframe to the new dataframe
df_mrm['Q3'] = start['Q3'] #    Assign the Q3 column from the start dataframe to the new dataframe
#change the Q1 and Q3 columns to float
df_mrm['Q1'] = df_mrm['Q1'].astype(float).round(0)
df_mrm['Q3'] = df_mrm['Q3'].astype(float).round(0)
#round the Q1 and Q3 m/z values to 1 decimal places
df_mrm['Q1'] = round(df_mrm['Q1'],1)
df_mrm['Q3'] = round(df_mrm['Q3'],1)
#Change column name Q1 to Parent_Ion and Q3 to Product_Ion


#For loop to create the transition column from the Q1 and Q3 columns
for index in range(len(df_mrm)):
    df_mrm.loc[index,'Transition'] = str(df_mrm.loc[index,'Q1']) + ' -> ' + str(df_mrm.loc[index,'Q3'])

df_mrm.rename(columns={'Q1':'Parent_Ion'}, inplace=True)
df_mrm.rename(columns={'Q3':'Product_Ion'}, inplace=True)
#df_mrm['Transition'] = lipid_file_mrm['Transition'] #Assign the Transition column from the lipid_file_mrm to the new dataframe



df_mrm.head()


/tmp/ipykernel_706/351916125.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(1)
/tmp/ipykernel_706/351916125.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Product_Ion'] = mrm_list_offical['Product_Ion'].round(1)
/tmp/ipykernel_706/351916125.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

                                Lipid  Parent_Ion  Product_Ion      Transition
0                            LPC(2:0)       300.1        184.1  300.1 -> 184.1
1                            LPC(3:1)       312.1        184.1  312.1 -> 184.1
2                  LPC(3:0),PC(O-3:0)       314.1        184.1  314.1 -> 184.1
3        LPC(4:0),PC(O-4:0),PC(O-5:0)       328.2        184.1  328.2 -> 184.1
4                    PC(4:0),LPC(5:0)       342.2        184.1  342.2 -> 184.1
5                            LPC(6:0)       356.2        184.1  356.2 -> 184.1
6                    PC(6:0),LPC(7:0)       370.2        184.1  370.2 -> 184.1
7                            LPC(8:0)       384.2        184.1  384.2 -> 184.1
8                         LPC(O-10:1)       396.3        184.1  396.3 -> 184.1
9                    PC(8:0),LPC(9:0)       398.2        184.1  398.2 -> 184.1
10               LPC(10:0),PC(O-10:0)       412.2        184.1  412.2 -> 184.1
11                 PC(10:0),LPC(11:0)       426.3   

,Lipid,Parent_Ion,Product_Ion,Transition
0,Hexacosanoyl carnitine,540.0,85.0,540.0 -> 85.0
1,Hexacosanoyl carnitine,540.0,60.0,540.0 -> 60.0
2,Hexacosanoyl carnitine,540.0,144.0,540.0 -> 144.0
3,O-behenoylcarnitine,484.0,144.0,484.0 -> 144.0
4,O-behenoylcarnitine,484.0,85.0,484.0 -> 85.0


In [47]:
search_list = pd.read_excel('./data_OzESI/search/search_list_v2.xlsx')
found_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Transition'])

search_list['Product_Ion'] = search_list['Parent_Ion'] - search_list['neutral_loss']
#Round search list columns to 0 deciaml places
search_list['Parent_Ion'] = search_list['Parent_Ion'].round(0)
search_list['Product_Ion'] = search_list['Product_Ion'].round(0)

print(search_list.head())

#Search for matches between mrm_list_offical and search_list for Parent Ion and Product Ion
for i in range(len(mrm_list_offical)):
    for j in range(len(search_list)):
        if mrm_list_offical.loc[i,'Parent_Ion'] == search_list.loc[j,'Parent_Ion'] and mrm_list_offical.loc[i,'Product_Ion'] == search_list.loc[j,'Product_Ion']:
           # print('yes')
            found_list = found_list.append(mrm_list_offical.loc[i,:])

# Do the same search for df_mrm and search_list for Parent Ion and Product Ion
for i in range(len(df_mrm)):
    for j in range(len(search_list)):
        if df_mrm.loc[i,'Parent_Ion'] == search_list.loc[j,'Parent_Ion'] and df_mrm.loc[i,'Product_Ion'] == search_list.loc[j,'Product_Ion']:
           # print('yes')
            found_list = found_list.append(df_mrm.loc[i,:])


# found_list.to_excel('./data_OzESI/data_csv/found_list.xlsx', index=False)      
found_list.head(25)


   Parent_Ion  neutral_loss  Product_Ion
0       821.0         299.2        522.0
1       823.0         299.2        524.0
2       849.0         299.2        550.0
3       851.0         299.2        552.0
4       873.0         299.2        574.0


/tmp/ipykernel_706/1225345404.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(df_mrm.loc[i,:])
/tmp/ipykernel_706/1225345404.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(df_mrm.loc[i,:])
/tmp/ipykernel_706/1225345404.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(df_mrm.loc[i,:])
/tmp/ipykernel_706/1225345404.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(df_mrm.loc[i,:])
/tmp/ipykernel_706/1225345404.py:23: FutureWarning: The frame.append method is deprecated and will be removed from panda

,Lipid,Parent_Ion,Product_Ion,Transition
1063,TAG(48:2)_FA 18:1,821.0,522.0,821.0 -> 522.0
1067,TAG(48:1)_FA 18:1,823.0,524.0,823.0 -> 524.0
1079,TAG(50:3)_FA 18:1,849.0,550.0,849.0 -> 550.0
1083,TAG(50:2)_FA 18:1,851.0,552.0,851.0 -> 552.0
1099,TAG(52:4)_FA 18:1,873.0,574.0,873.0 -> 574.0
1103,TAG(52:3)_FA 18:1,875.0,576.0,875.0 -> 576.0
1107,TAG(52:2)_FA 18:1,877.0,578.0,877.0 -> 578.0
1111,TAG(52:1)_FA 18:1,879.0,580.0,879.0 -> 580.0
1131,TAG(54:5)_FA 18:1,899.0,600.0,899.0 -> 600.0
1135,TAG(54:4)_FA 18:1,901.0,602.0,901.0 -> 602.0


In [48]:
df_OzESI = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

for i in range(3,21):
    df_OzESI.loc[i,'DB_Position'] = i
    df_OzESI.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3))

df_OzESI.head(25)


,DB_Position,Aldehyde_Ion
3,3,26
4,4,40
5,5,54
6,6,68
7,7,82
8,8,96
9,9,110
10,10,124
11,11,138
12,12,152


In [49]:
OzESI_list = [7,9,12,14]
#create columns in found list called n-# based on the numbers in OzESI_list
for i in OzESI_list:
    found_list['n-' + str(i)] = ''
print(found_list.head(25))

# Parent_Ion - Aldehyde_Ion *

# Subtract the aldehyde ion from the parent ion with the corresponding DB position and add the result to the found_list dataframe
for i in range(3,len(df_OzESI)):
    for j in range(len(found_list)):
        #if df_OzESI.loc[i,'DB_Position']:
        for k in range(4,8):
            if df_OzESI.loc[i,'DB_Position'] == OzESI_list[k-4]:
                found_list.iloc[j,k] = found_list.iloc[j,1] - df_OzESI.loc[i,'Aldehyde_Ion'] + 1
                print(found_list.head(5))
                print(i)
                print(df_OzESI.loc[i,'Aldehyde_Ion'])
                print(j)
                print(found_list.iloc[j,1])
                print(found_list.iloc[j,1] - df_OzESI.loc[i,'Aldehyde_Ion'])
                print('----')
                # print(found_list.iloc[j,count])
                # print(found_list.iloc[j,1])
                # print(df_OzESI.loc[i,'Aldehyde_Ion'])
                # print(df_OzESI.loc[i,'DB_Position'])
  #5
                #print('yes')
        #else:
            #print('no')
    # if df_OzESI.loc[i,'DB_Position'] in OzESI_list:
    #     print('yes')
        
found_list.head(25)
#         result = found_list.loc[i,'Parent_Ion'] - df_OzESI.loc[i,'Aldehyde_Ion']

#         #found_list['n-' + str(df_OzESI.loc[i,'DB_Position'])] = found_list.loc[i,'Parent_Ion'] - df_OzESI.loc[i,'Aldehyde_Ion']
# found_list.head(25)

# #Subtract the aldehyde ion from the parent ion. Select the rows in df_OzESI where the DB_Position is in the OzESI_list
# for parent in found_list['Parent_Ion']:
#     for i in range(len(df_OzESI)):
#         if df_OzESI.loc[i,'DB_Position'] in OzESI_list:
#             #create column in found_list called OzESI with DB position and aldehyde ion
#             found_list['OzESI'] = df_OzESI.loc[i,'DB_Position'].astype(str) + ' ' + df_OzESI.loc[i,'Aldehyde_Ion'].astype(str)
    

                   Lipid  Parent_Ion  Product_Ion      Transition n-7 n-9  \
1063   TAG(48:2)_FA 18:1       821.0        522.0  821.0 -> 522.0           
1067   TAG(48:1)_FA 18:1       823.0        524.0  823.0 -> 524.0           
1079   TAG(50:3)_FA 18:1       849.0        550.0  849.0 -> 550.0           
1083   TAG(50:2)_FA 18:1       851.0        552.0  851.0 -> 552.0           
1099   TAG(52:4)_FA 18:1       873.0        574.0  873.0 -> 574.0           
1103   TAG(52:3)_FA 18:1       875.0        576.0  875.0 -> 576.0           
1107   TAG(52:2)_FA 18:1       877.0        578.0  877.0 -> 578.0           
1111   TAG(52:1)_FA 18:1       879.0        580.0  879.0 -> 580.0           
1131   TAG(54:5)_FA 18:1       899.0        600.0  899.0 -> 600.0           
1135   TAG(54:4)_FA 18:1       901.0        602.0  901.0 -> 602.0           
1139   TAG(54:3)_FA 18:1       903.0        604.0  903.0 -> 604.0           
1143   TAG(54:2)_FA 18:1       905.0        606.0  905.0 -> 606.0           

,Lipid,Parent_Ion,Product_Ion,Transition,n-7,n-9,n-12,n-14
1063,TAG(48:2)_FA 18:1,821.0,522.0,821.0 -> 522.0,740.0,712.0,670.0,642.0
1067,TAG(48:1)_FA 18:1,823.0,524.0,823.0 -> 524.0,742.0,714.0,672.0,644.0
1079,TAG(50:3)_FA 18:1,849.0,550.0,849.0 -> 550.0,768.0,740.0,698.0,670.0
1083,TAG(50:2)_FA 18:1,851.0,552.0,851.0 -> 552.0,770.0,742.0,700.0,672.0
1099,TAG(52:4)_FA 18:1,873.0,574.0,873.0 -> 574.0,792.0,764.0,722.0,694.0
1103,TAG(52:3)_FA 18:1,875.0,576.0,875.0 -> 576.0,794.0,766.0,724.0,696.0
1107,TAG(52:2)_FA 18:1,877.0,578.0,877.0 -> 578.0,796.0,768.0,726.0,698.0
1111,TAG(52:1)_FA 18:1,879.0,580.0,879.0 -> 580.0,798.0,770.0,728.0,700.0
1131,TAG(54:5)_FA 18:1,899.0,600.0,899.0 -> 600.0,818.0,790.0,748.0,720.0
1135,TAG(54:4)_FA 18:1,901.0,602.0,901.0 -> 602.0,820.0,792.0,750.0,722.0


In [50]:
i = 0
while i < 100:
    if not os.path.exists('./data_OzESI/data_excel/found_list.xlsx'):
        found_list.to_excel('./data_OzESI/data_excel/found_list.xlsx', index=False)
        break
    elif not os.path.exists('./data_OzESI/data_excel/found_list' + str(i) + '.xlsx'):
        found_list.to_excel('./data_OzESI/data_excel/found_list' + str(i) + '.xlsx', index=False)
        break
    else:
        i += 1
